### Task 1

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
class KMeansClustering:
    
    def __init__(self, k, stopping_criterion="no_change") -> None:
        self.k = k
        self.stopping_criterion = stopping_criterion
        self.centroids = None
        self._sse_score = None
        self._last_sse_score = float('inf')
        self._iterations = 0

    def euclidean_distance(self, data_point, centroids):
        return np.sqrt(np.sum((centroids - data_point)**2, axis=1))

    def __sum_of_squared_errors_calc(self, centroids, data, y):
        sum_of_errors = 0.0
        for idx, d in enumerate(data):
            sum_of_errors += np.sum((centroids[y[idx]] - d) ** 2)

        return sum_of_errors

    def get_sum_of_squared_error(self):
        return self._sse_score
    def get_iterations_to_converge(self):
        return self._iterations
    
    def fit(self, X, max_iterations=200):
        self.centroids = np.random.uniform(
            low=np.amin(X, axis=0),
            high=np.amax(X, axis=0),
            size=(self.k, X.shape[1]))

        y = []
        for _ in range(max_iterations):
            y = []
            for data_point in X:

                distances = self.euclidean_distance(
                    data_point=data_point,
                    centroids=self.centroids)
                # print(distances.shape)
                cluster_num = np.argmin(distances)
                y.append(cluster_num)
            y = np.asarray(y)

            cluster_indices = []

            for idx in range(self.k):
                cluster_indices.append(np.argwhere(y == idx))

            cluster_centers = []

            for i, indices in enumerate(cluster_indices):
                if len(indices) == 0:
                    cluster_centers.append(self.centroids[i])
                else:
                    cluster_centers.append(np.mean(X[indices], axis=0)[0])

            if self.stopping_criterion == "no_change" and np.max(self.centroids - np.array(cluster_centers)) < 1e-3:
                break
            elif self.stopping_criterion == "increase_sse":
                current_sse = self.__sum_of_squared_errors_calc(X, np.array(cluster_centers), y)
                if current_sse > self._last_sse_score:
                    break
                self._last_sse_score = current_sse
            else:
                self.centroids = np.array(cluster_centers)
            self._iterations += 1

        # Calculate the final SSE after performing K-means
        self._sse_score = self.__sum_of_squared_errors_calc(X, self.centroids, y)
        
        return y

In [4]:
data = np.array(pd.read_csv('datasets/kmeans_data/data.csv', header=None))
labels = np.ravel(pd.read_csv('datasets/kmeans_data/label.csv', header=None))
print('Data: ', data.shape)
print('Labels: ', labels.shape)

Data:  (10000, 784)
Labels:  (10000,)


In [5]:
unique_labels = np.unique(labels)
no_of_clusters = unique_labels.size
MAX_ITERATIONS = 100

In [6]:
euclidean_kmeans_m = KMeansClustering(k=no_of_clusters)
euclidean_kmeans_m_labels = euclidean_kmeans_m.fit(X=data, max_iterations=MAX_ITERATIONS)

In [7]:
cosine_distances = pairwise_distances(data, metric='cosine')
cosine_kmeans_m = KMeansClustering(k=no_of_clusters)
cosine_kmeans_m_labels = cosine_kmeans_m.fit(cosine_distances, max_iterations=MAX_ITERATIONS)

In [8]:
jaccard_distances = pairwise_distances(data, metric='hamming')
jaccard_kmeans_m = KMeansClustering(k=no_of_clusters)
jaccard_kmeans_m_labels = jaccard_kmeans_m.fit(X=jaccard_distances, max_iterations=MAX_ITERATIONS)

#### Q1. Run K-means clustering with Euclidean, Cosine and Jarcard similarity. Specify K= the number of categorical values of y (the number of classifications). Compare the SSEs of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method is better?

From the SSE values, the one with the lowest SSE is Jaccard-K-means. So using Jaccard K-Means is betterwith the lowest SSE of 1059.83.

In [9]:
sse_euclidean_m = euclidean_kmeans_m.get_sum_of_squared_error()
sse_euclidean_m

53414304.183158375

In [10]:
sse_cosine_m = cosine_kmeans_m.get_sum_of_squared_error()
sse_cosine_m

3794.605548285986

In [11]:
see_jaccard_m = jaccard_kmeans_m.get_sum_of_squared_error()
see_jaccard_m

1515.631195969257

#### Q2. Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. First, label each cluster using the majority vote label of the data points in that cluster. Later, compute the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which metric is better? (10 points)

Based on the accuracy computation of majority vote, Euclidean accuracy seems to perform better.

In [12]:
def label_clusters(labels, true_labels):
    unique_labels = np.unique(true_labels)
    cluster_labels = np.zeros(len(labels), dtype=np.int)
    for cluster in range(no_of_clusters):
        cluster_indices = np.where(labels == cluster)[0]
        cluster_true_labels = true_labels[cluster_indices]
        majority_label = np.argmax([np.sum(cluster_true_labels == label) for label in unique_labels])
        cluster_labels[cluster_indices] = majority_label
    return cluster_labels

# Label clusters using majority vote
cluster_labels_euclidean = label_clusters(euclidean_kmeans_m_labels, labels)
cluster_labels_cosine = label_clusters(cosine_kmeans_m_labels, labels)
cluster_labels_jaccard = label_clusters(jaccard_kmeans_m_labels, labels)

# Compute predictive accuracy
accuracy_euclidean = accuracy_score(labels, cluster_labels_euclidean)
accuracy_cosine = accuracy_score(labels, cluster_labels_cosine)
accuracy_jaccard = accuracy_score(labels, cluster_labels_jaccard)

print("Accuracy Euclidean-K-means:", accuracy_euclidean)
print("Accuracy Cosine-K-means:", accuracy_cosine)
print("Accuracy Jaccard-K-means:", accuracy_jaccard)

Accuracy Euclidean-K-means: 0.5924
Accuracy Cosine-K-means: 0.4826
Accuracy Jaccard-K-means: 0.28


#### Q3: Set up the same stop criteria: “when there is no change in centroid position OR when the SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-Kmeans, Jarcard-K-means. Which method requires more iterations and times to converge? (10 points)

Note : Max iterations have been set to 500 and the change in centroid position is less than 1e-3.

From the results below we see that Cosine-K means takes 57 iterations to converge hence the Cosine-K means takes more time to converge followed by Euclidean-K means and then Jaccard-K means.

In [13]:
print("Iterations to converge for Euclidean-K-means =", euclidean_kmeans_m.get_iterations_to_converge())
print("Iterations to converge for Cosine-K-means =", cosine_kmeans_m.get_iterations_to_converge())
print("Iterations to converge for Jaccard-K-means =", jaccard_kmeans_m.get_iterations_to_converge())

Iterations to converge for Euclidean-K-means = 76
Iterations to converge for Cosine-K-means = 13
Iterations to converge for Jaccard-K-means = 5


**Q4:** Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to 
the following three terminating conditions: (10 points)
  - when there is no change in centroid position
  - when the SSE value increases in the next iteration
  - when the maximum preset value (e.g., 100) of iteration is complete

##### (a) Check for SSE for Euclidean K-means, Cosine K-means and Jacard K-means when there is no change in centroid position

In [14]:
euclidean_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
euclidean_kmeans_1_predicted_labels = euclidean_kmeans_1.fit(data, max_iterations=MAX_ITERATIONS)
print('SSE of Euclidean K-means when there is no change in centroid position =', euclidean_kmeans_1.get_sum_of_squared_error())

SSE of Euclidean K-means when there is no change in centroid position = 69805396.6262142


In [15]:
cosine_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
cosine_kmeans_1_predicted_labels = cosine_kmeans_1.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
print('SSE of Cosine K-means when there is no change in centroid position =', cosine_kmeans_1.get_sum_of_squared_error())

SSE of Cosine K-means when there is no change in centroid position = 6684.063186410751


In [16]:
jaccard_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
jaccard_kmeans_1_predicted_labels = jaccard_kmeans_1.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
print('SSE of Jaccard K-means when there is no change in centroid position =', jaccard_kmeans_1.get_sum_of_squared_error())

SSE of Jaccard K-means when there is no change in centroid position = 2011.7288898880395


##### (b) Check for SSE for Euclidean K-means, Cosine K-means and Jacard K-means when the SSE value increases in the next iteration

In [17]:
euclidean_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
euclidean_kmeans_2_predicted_labels = euclidean_kmeans_2.fit(data, max_iterations=MAX_ITERATIONS)
print('SSE of Euclidean K-means when the SSE value increases in the next iteration =',
      euclidean_kmeans_2.get_sum_of_squared_error())

SSE of Euclidean K-means when the SSE value increases in the next iteration = 126961038.58517787


In [18]:
cosine_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
cosine_kmeans_2_predicted_labels = cosine_kmeans_2.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
print('SSE of Cosine K-means when the SSE value increases in the next iteration =',
      cosine_kmeans_2.get_sum_of_squared_error())

SSE of Cosine K-means when the SSE value increases in the next iteration = 11480.31721225982


In [19]:
jaccard_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
jaccard_kmeans_2_predicted_labels = jaccard_kmeans_2.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
print('SSE of Jaccard K-means when the SSE value increases in the next iteration =',
      jaccard_kmeans_2.get_sum_of_squared_error())

SSE of Jaccard K-means when the SSE value increases in the next iteration = 2512.3304722666135


##### (c) Check for SSE for Euclidean K-means, Cosine K-means and Jacard K-means when the maximum preset value (e.g., 100) of iteration is complete

In [20]:
euclidean_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
euclidean_kmeans_3_predicted_labels = euclidean_kmeans_3.fit(data, max_iterations=MAX_ITERATIONS)
print(f'SSE of Euclidean K-means when the maximum preset value {MAX_ITERATIONS} is complete =',
      euclidean_kmeans_3.get_sum_of_squared_error())

SSE of Euclidean K-means when the maximum preset value 100 is complete = 35280045.88769699


In [21]:
cosine_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
cosine_kmeans_3_predicted_labels = cosine_kmeans_3.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
print(f'SSE of Cosine K-means when the maximum preset value {MAX_ITERATIONS} is complete =',
      cosine_kmeans_3.get_sum_of_squared_error())

SSE of Cosine K-means when the maximum preset value 100 is complete = 5157.762447672464


In [ ]:
jaccard_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
jaccard_kmeans_3_predicted_labels = jaccard_kmeans_3.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
print(f'SSE of Jaccard K-means when the maximum preset value {MAX_ITERATIONS} is complete =',
      jaccard_kmeans_3.get_sum_of_squared_error())